Proyecto (primer entregable)

Francisco Velasco Medina 165473 y 
José Luis Cordero Rodríguez 164860

En esta primera parte del proyecto se estudiará e implementará el algoritmo de Búsqueda Lineal de Newton con modificación a la Hessiana. 


In [4]:
#Importar librerías 
import numpy as np
import math as ma
import random as rd
from numpy import linalg as la
from numpy.random import seed 
from numpy.random import rand

Empezamos con el código para calcular el gradiente. El siguiente método devuelve el gradiente
de una función tomando como parámetros a la función (regla de correspondencia) y un punto. 

In [5]:
#Código para caluclar el gradiente
#elegimos un error apropiado
def grad(f,xo):
    n = len(xo)
    eps=0.000001
    res=np.zeros(n)
    for i in range(n):
        zer = np.zeros(n)
        zer[i] += eps
        x1 = xo + zer
        res[i] = (f(x1)-f(xo))/eps
    return res

Se crean tres funciones diferentes, cada una de ellos con una parametrización distinta. Para
la primera, a = 1 
y b = 100. En la segunda a = 2 y b = 20. Finalmente, en la última, a = 0 y b = 50.  

In [ ]:
#Definimos la función Rosenbrock de forma parametrizada 
xo=[1,1]
def Rosen(a, b):
  return lambda x: (a - x[0]) ** 2 + b * (x[1] - x[0] ** 2) ** 2

uno_cien = Rosen(1, 100)

dos_veinte = Rosen(2, 20)

cero_cincuenta = Rosen(0, 50)


In [21]:
#Calculamos el gradiente de forma anlítica y lo calcula bien 
grad(uno_cien,xo)

array([4.010004e-04, 1.000000e-04])

En el siguiente chunk está el método que se utiliza para calcular la Hessiana de una función
especificada. Para su desarrollo se usaron diferencias finitas. El método toma como argumento
una función y un punto dado. 

In [22]:
#Código que calcula la hessiana
#el desarrollo de este método se hará con diferencias finitas 
def hess(f,xo):
    n = len(xo)
    eps=0.000001
    res=np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            zer = np.zeros(n)
            zer2 = np.zeros(n)
            zer[i] += eps
            zer2[j] += eps
            x_e = xo + zer + zer2
            x_ei = xo + zer
            x_ej = xo + zer2
            res[i][j] = (f(x_e)-f(x_ei)-f(x_ej)+f(xo))/(eps**2)
    return res

In [23]:
#imprimimos la Hessiana y verificamos que es correcto el cálculo  
hess(uno_cien,xo)

array([[ 802.00239973, -400.00019995],
       [-400.00019995,  199.99999997]])

A continuación construimos el método que verifica que las condiciones de optimalidad se 
cumplen. 

In [25]:
def positiva_def(A):
    propios, vectores = la.eig(A)
    for L in propios:
        if L.real <= 0:
            return False
    return True

def grad_nulo(grad,tol):
    print(grad)
    for x in grad:
      if abs(x) > tol:
        return False
    return True

def condiciones_optimalidad(f, xk,tol):
    # Código que regresa si el punto xk cumple 
    # con las condiciones de optimalidad.
    # La matriz hessiana debe ser positiva definida, 
    # en otras palabras sus valores propios deben ser positivos.
    return positiva_def(hess(f,xk)) and grad_nulo(grad(f,xk),tol)

In [27]:
#permitimos que el usuario ingrese un nivel de tolerancia 
condiciones_optimalidad(uno_cien,xo,.001)

[4.010004e-04 1.000000e-04]


True

In [28]:
# definir función mk 
def mk(f,xo,p):
    H = hess(f,xo)
    G = grad(f,xo)
    pt = np.transpose(p)
    return f(xo) + np.dot(pt,G) + 0.5 * np.dot(np.dot(pt,H),pt)

In [35]:
#probamos con un punto la función
p=[.01,.02]
mk(uno_cien,xo,p)

0.00010608999340135658

In [36]:
#Algoritmo de búsqueda de paso (BacksS)
def BackS(a,f,xk,pk):
    rd.seed(123)
    c = rd.uniform(0,1)
    while f(xk + a * pk) > f(xk) + c * a * np.dot(grad(f,xk),pk):
        rho = rd.uniform(0,1)
        a = rho*a
    return a 

In [51]:
BackS(1,uno_cien,np.array([1.2,1.2]),np.array([-1,-2]))

0.08718667752263232

In [38]:
#Algoritmo Cholesky con mútilplo de la identidad
#np.transpose
def Cholesky(A,b,k):
    t = 0
    if min(np.diag(A)) > 0:
         t = 0
    else: 
        t = -min(np.diag(A)) + b
        
    for j in range(k):
        try: 
            L = la.cholesky(A + t*np.identity(len(A)))
        except:
            t = max(2*t,b)
        else:
            break
    return np.dot(L,L)
    

In [39]:
#Algoritmo de búsqueda lineal de Newton Modificado
def Newton_Mod(xk,f,n,k,b):
    for i in range(n):
        Bk = hess(f,xk)
        try:
            L = la.cholesky(Bk)
        except:
            Bk = Cholesky(Bk,b,k)
        pk = np.dot(la.inv(Bk),-1 * grad(f,xk))
        xk = xk + BackS(1,f,xk,pk)* pk
    return xk
        

Para probar que nuestro código funcione, intentamos con las tres funciones creadas y un punto
inicial. Notemos que la función Rosenrbock estándar, es decir, de parámetros 1 y 100 
respectivamente tiene un mínimo global en (1,1). Depués de varias iteraciones (1000), el
algoritmo logra aproximarse al punto con bastante precisión. Lo mismo sucede con los otros
ejemplos. 

In [60]:
print("El punto que minimiza (alcanzado por el algoritmo) es:")
(x,y)=Newton_Mod([-1.2,1.2],uno_cien,1000,1000,1)
print("(x,y) = ", ('%0.3f' % x,'%0.3f' % y ))

El punto que minimiza (alcanzado por el algoritmo) es:
(x,y) =  ('1.000', '0.999')


In [61]:
print("El punto que minimiza (alcanzado por el algoritmo) es:")
(x,y)=Newton_Mod([1.2,1.2],dos_veinte,1000,1000,1)
print("(x,y) = ", ('%0.3f' % x,'%0.3f' % y ))

El punto que minimiza (alcanzado por el algoritmo) es:
(x,y) =  ('2.000', '3.999')


In [62]:
print("El punto que minimiza (alcanzado por el algoritmo) es:")
(x,y)=Newton_Mod([1.2,1.2],cero_cincuenta,1000,1000,1)
print("(x,y) = ", ('%0.3f' % x,'%0.3f' % y ))

El punto que minimiza (alcanzado por el algoritmo) es:
(x,y) =  ('-0.000', '-0.000')


Se crean tres funciones cuyos parámetros se calculan de forma aleatoria. 

In [47]:
seed(1)
values=rand(6)
azar1=Rosen(100*values[0],100*values[1])
azar2=Rosen(-50*values[2],10*values[3]+5)
azar3=Rosen(values[4]+6,50*values[5])

In [57]:
print("El punto que minimiza (alcanzado por el algoritmo) es:")
(x,y)=Newton_Mod([1.2,1.2],azar1,1000,1000,1)
print("(x,y) = ", ('%0.3f' % x,'%0.3f' % y ))

El punto que minimiza (alcanzado por el algoritmo) es:
(x,y) =  ('12.725', '161.944')


In [58]:
print("El punto que minimiza (alcanzado por el algoritmo) es:")
(x,y)=Newton_Mod([1.2,1.2],azar2,1000,1000,1)
print("(x,y) = ", ('%0.3f' % x,'%0.3f' % y ))

El punto que minimiza (alcanzado por el algoritmo) es:
(x,y) =  ('-0.006', '0.000')


In [59]:
print("El punto que minimiza (alcanzado por el algoritmo) es:")
(x,y)=Newton_Mod([1.2,1.2],azar3,1000,1000,1)
print("(x,y) = ", ('%0.3f' % x,'%0.3f' % y ))

El punto que minimiza (alcanzado por el algoritmo) es:
(x,y) =  ('6.146', '37.778')
